In [1]:
!unzip '/content/drive/MyDrive/MP_DATA.zip' -d '/content/drive/MyDrive/'


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install mediapipe

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)  #33*4 extra 1 is for visibilty only pose has this
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)    #468*3
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)  #21 keypoints and 3 dimension x,y,z
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])


In [9]:
#SETTING DIRECTORY
#Path for exported_data i.e numpy arrays
DATA_PATH = os.path.join('MP_DATA')

#subsequent actions
actions = np.array(['Helloh','Fuck','Noice','ape hamuduruvane','Okay'])

no_sequences = 30  # 30 number of vids of data
sequence_length = 30 #each of 30frames per sec in length


In [10]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):


              # Read feed
              ret, frame = cap.read()

              # Make detections
              image, results = mediapipe_detection(frame, holistic)
              print(results)

              # Draw landmarks
              draw_styled_landmarks(image, results)

              #collection logic with breaks
              if frame_num ==  0:         #0th frame
               cv2.putText(image,'STARTING COLLECTION',(120,200),
                           cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),4,cv2.LINE_AA)
               cv2.putText(image,'Collecting frames for {} Video number{}'.format(action,sequence),(15,12),
                           cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
              else:
                 cv2.putText(image,'Collecting frames for {} Video Number{}'.format(action,sequence),(15,12),
                             cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)

                 #export keypoints
              keypoints = extract_keypoints(results)
              npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
              np.save(npy_path,keypoints)

              # Show to screen
              cv2.imshow('OpenCV Feed', image)

            # Break gracefully
              if cv2.waitKey(10) & 0xFF == ord('q'):
                 break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [12]:
!pip install scikit-learn

In [13]:
cap.release()
cv2.destroyAllWindows()

In [14]:
!pip install tensorflow

In [15]:
from tensorflow.keras.utils import to_categorical

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
label_map = {label:num for num,label in enumerate(actions)}

In [18]:
cd /content/drive/MyDrive

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive'
E:\github\on test\HandGestureRecog-master


In [19]:
sequences,labels = [],[]
for action in actions:
  for sequence in range(no_sequences):
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
      window.append(res)
    sequences.append(window)
    labels.append(label_map[action])

In [20]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [21]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.05)

**LSTM Neural Network**

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu', input_shape=(30,1662)))   #30frames, 1662 keypoints
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))


In [26]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy']) #for multi class use categ cross entropy

In [ ]:
model.fit(X_train,y_train,epochs=1500,callbacks=[tb_callback])

Epoch 1/1500
6/6 [==============================] - 4s 54ms/step - loss: 9.9899 - categorical_accuracy: 0.1462
Epoch 2/1500
6/6 [==============================] - 0s 65ms/step - loss: 2.3515 - categorical_accuracy: 0.1930
Epoch 3/1500
6/6 [==============================] - 0s 54ms/step - loss: 3.3771 - categorical_accuracy: 0.1579
Epoch 4/1500
6/6 [==============================] - 0s 54ms/step - loss: 5.6516 - categorical_accuracy: 0.1462
Epoch 5/1500
6/6 [==============================] - 0s 65ms/step - loss: 4.4744 - categorical_accuracy: 0.1696
Epoch 6/1500
6/6 [==============================] - 0s 66ms/step - loss: 8.8176 - categorical_accuracy: 0.1813
Epoch 7/1500
6/6 [==============================] - 0s 62ms/step - loss: 8.9494 - categorical_accuracy: 0.1871
Epoch 8/1500
6/6 [==============================] - 0s 65ms/step - loss: 8.7946 - categorical_accuracy: 0.1696
Epoch 9/1500
6/6 [==============================] - 0s 62ms/step - loss: 26.9364 - categorical_accuracy: 0.1871


**Make Predictions**

In [ ]:
res= model.predict[X_test]

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

**Save Model**

In [ ]:
model.save('MediapipeModel.h5')

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
yhat = model.predict(X_test)

In [ ]:
yactual= np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(yactual,yhat)

In [ ]:
accuracy_score(ytrue,yhat)

In [ ]:
# Update the colors list with two more colors
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (255, 0, 0), (0, 255, 0),(255,165,0)]

# Function to visualize probabilities
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
from scipy import stats

**Test in Real time**

In [ ]:
sequence=[]
sentence=[]
threshold = 0.4

cap = cv2.VideoCapture(2)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret,frame = cap.read()

    image,results = mediapipe_detection(frame,holistic)
    print(results)

    draw_styled_landmarks(image,results)

    #Prediction logic code
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[:-30]

    if len(sequence) ==30:
      res = model.predict(np.expand_dims(sequence,axis=0))[0]
      print(actions[np.argmax(res)])


      #Visulaisation prediction logic
    if res[np.argmax(res)] > threshold:
      if len(sentence) > 0:
        if actions[np.argmax(res)] != sentence[-1]:
          sentence.append(actions[np.argmax(res)])
      else:
          sentence.append(actions[np.argmax(res)])
    if len(sentence) > 5:
      sentence = sentence[-5:]

    #Visualise Probability
    image = prob_viz(res,actions,image,actions)
    # cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image, ''.join(sentence),(3,30),
                  cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('OpenCV Feed',image)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('C:/Users/jaikr/Downloads/MediapipeModel.h5')

# Define the threshold for prediction
threshold = 0.4

# Define your functions for visualization and other tasks
# (mediapipe_detection, draw_styled_landmarks, extract_keypoints, prob_viz)

# Start capturing video from the webcam
cap = cv2.VideoCapture(2)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results)

        # Prediction logic code
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Maintain the last 30 frames

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            # Visualization prediction logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]

            # # Visualize Probability
            # image = prob_viz(res, actions, image, colors)
            # cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            # cv2.putText(image, ''.join(sentence), (3, 30),
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
 #Load the trained model
# model = load_model('C:/Users/jaikr/Downloads/MediapipeModel.h5')
sequence=[]
sentence=[]
# Define the threshold for prediction
threshold = 0.4

# Define your functions for visualization and other tasks
# (mediapipe_detection, draw_styled_landmarks, extract_keypoints, prob_viz)

# Start capturing video from the webcam
cap = cv2.VideoCapture(2)



with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret,frame = cap.read()

    image,results = mediapipe_detection(frame,holistic)
    print(results)

    draw_styled_landmarks(image,results)

    #Prediction logic code
    keypoints = extract_keypoints(results)
    sequence.insert(keypoints)
    sequence = sequence[:-30]

    if len(sequence) ==30:
      res = model.predict(np.expand_dims(sequence,axis=0))[0]
      print(actions[np.argmax(res)])


      #Visulaisation prediction logic
    if res[np.argmax(res)] > threshold:
      if len(sentence) > 0:
        if actions[np.argmax(res)] != sentence[-1]:
          sentence.append(actions[np.argmax(res)])
      else:
          sentence.append(actions[np.argmax(res)])
    if len(sentence) > 5:
      sentence = sentence[-5:]

    #Visualise Probability
    image = prob_viz(res,actions,image,actions)
    cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image, ''.join(sentence),(3,30),
                  cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('OpenCV Feed',image)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(2)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()